In [1]:
!pip install gdown

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Using cached requests-2.24.0-py2.py3-none-any.whl (61 kB)
Processing /home/ubuntu/.cache/pip/wheels/66/13/60/ef107438d90e4aad6320e3424e50cfce5e16d1e9aad6d38294/filelock-3.0.12-cp27-none-any.whl
     |████████████████████████████████| 70 kB 900 kB/s eta 0:00:01
  Using cached certifi-2020.6.20-py2.py3-none-any.whl (156 kB)
     |████████████████████████████████| 127 kB 2.7 MB/s eta 0:00:01
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Using cached chardet-3.0.4-py2.py3-

In [2]:
!gdown --id 1uRgJ5MzqoGh-XYQUFAVBUGQYlLi7aMXx

Downloading...
From: https://drive.google.com/uc?id=1uRgJ5MzqoGh-XYQUFAVBUGQYlLi7aMXx
To: /home/ubuntu/jsl/streamlit-demo-apps/new_notebook/absa_sparknlp/public_app/notebooks/ABSA_glove_absa.zip
83.9MB [00:06, 12.8MB/s]


In [3]:
!unzip ABSA_glove_absa.zip

Archive:  ABSA_glove_absa.zip
   creating: ABSA_glove_absa/
  inflating: ABSA_glove_absa/.tensorflow.crc  
   creating: ABSA_glove_absa/fields/
   creating: ABSA_glove_absa/fields/datasetParams/
 extracting: ABSA_glove_absa/fields/datasetParams/_SUCCESS  
 extracting: ABSA_glove_absa/fields/datasetParams/.part-00000.crc  
  inflating: ABSA_glove_absa/fields/datasetParams/part-00003  
  inflating: ABSA_glove_absa/fields/datasetParams/part-00000  
 extracting: ABSA_glove_absa/fields/datasetParams/.part-00004.crc  
 extracting: ABSA_glove_absa/fields/datasetParams/.part-00007.crc  
  inflating: ABSA_glove_absa/fields/datasetParams/part-00001  
  inflating: ABSA_glove_absa/fields/datasetParams/part-00005  
 extracting: ABSA_glove_absa/fields/datasetParams/._SUCCESS.crc  
 extracting: ABSA_glove_absa/fields/datasetParams/.part-00003.crc  
 extracting: ABSA_glove_absa/fields/datasetParams/.part-00002.crc  
  inflating: ABSA_glove_absa/fields/datasetParams/part-00007  
 extracting: ABSA_glove

In [4]:
# -*- coding: utf-8 -*-

import os
import json
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['PATH'] = os.environ['JAVA_HOME'] + "/bin:" + os.environ['PATH']

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline

from sparknlp.training import CoNLL
import pyspark.sql.functions as F

# Start Spark session
spark = sparknlp.start(gpu=True)

In [5]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

token = Tokenizer()\
    .setInputCols(['document'])\
    .setOutputCol('token')

glove_embeddings = WordEmbeddingsModel.pretrained("glove_840B_300", "xx")\
    .setInputCols(["document", "token"])\
    .setOutputCol("embeddings")
    
loaded_ner_model = NerDLModel.load("ABSA_glove_absa")\
    .setInputCols(["document", "token", "embeddings"])\
    .setOutputCol("absa")

converter = NerConverter()\
    .setInputCols(["document", "token", "absa"])\
    .setOutputCol("absa_span")

ner_prediction_pipeline = Pipeline(
    stages = [
        document,
        token,
        glove_embeddings,
        loaded_ner_model,
        converter])

empty_data = spark.createDataFrame([['']]).toDF("text")
prediction_model = ner_prediction_pipeline.fit(empty_data)
sent_pipeline = Pipeline(
    stages = [document, sentence]
)

glove_840B_300 download started this may take some time.
Approximate size to download 2.3 GB
[OK!]


In [6]:
input_files = os.listdir('../inputs/')

In [9]:
input_paths = [os.path.join('../inputs/', file) for file in input_files]
output_paths = [os.path.join('../outputs/', file.replace('txt', 'csv')) for file in input_files]

In [8]:
input_paths

['../inputs/Example10.txt',
 '../inputs/Example4.txt',
 '../inputs/Example5.txt',
 '../inputs/Example3.txt',
 '../inputs/Example7.txt',
 '../inputs/Example9.txt',
 '../inputs/Example1.txt',
 '../inputs/Example8.txt',
 '../inputs/Example2.txt',
 '../inputs/Example6.txt']

In [15]:
for in_path, out_path in zip(input_paths, output_paths):
    text = open(in_path).read()
    df = spark.createDataFrame(pd.DataFrame({'text': [text]}))
    df1 = prediction_model.transform(df).toPandas()
    df2 = sent_pipeline.fit(empty_data).transform(df).toPandas()

    all_sents = df2['sentence'][0]

    sentences = []
    aspects = []
    sentiments = []
    for result in df1['absa_span'][0]:
        start, end = result['begin'], result['end']
        for sent in all_sents:
            if sent['begin'] <= start and sent['end'] >= end:
                sentences.append(sent['result'])
        aspects.append(result['result'])
        sentiment = "positive" if result['metadata']['entity'] == "POS" else "negative"
        sentiments.append(sentiment)
    final_result = pd.DataFrame.from_dict({"sentence": sentences, "aspect": aspects, "sentiment": sentiments})
    final_result.to_csv(out_path, index=False)

In [16]:
!ls ../outputs

Example1.csv   Example2.csv  Example4.csv  Example6.csv  Example8.csv
Example10.csv  Example3.csv  Example5.csv  Example7.csv  Example9.csv


In [19]:
df = pd.read_csv("../outputs/Example10.csv")

In [20]:
df.head()

,sentence,aspect,sentiment
0,We loved our Thai-style main which amazing wit...,Thai-style main,positive
1,We loved our Thai-style main which amazing wit...,lots of flavours,positive
2,But the service was below average and the chip...,service,negative
3,But the service was below average and the chip...,chips,negative
